# Getting started with __htlogicalgates__

Welcome to `htlogicalgates`. This notebook shows you how to create hardware-tailored logical circuits for stabilizer codes using our package.

## Installation

This package can be installed conveniently through `pip`.

In [ ]:
%pip install htlogicalgates

Alternatively, you can clone the [repository](https://github.com/erkue/htlogicalgates) and include the source files in your project. 

Now we can import the module:

In [1]:
import htlogicalgates as htlg

## How to tailor logical circuits

The main workflow for tailoring a circuit starts with creating three central objects:

In [2]:
stab_code = htlg.StabilizerCode("4_2_2")
connectivity = htlg.Connectivity("circular", num_qubits=4)
logical_gate = htlg.Circuit(2)
logical_gate.h(0)

After importing the package, we create three objects:

- First, we create a `StabilizerCode` for which we wish to find a logical circuit. In this example, we pass `"4_2_2"`, which selects the $⟦4,2,2⟧$ color code. Some common codes are predefined (see below), but custom codes can also be specified through a set of stabilizer generators. 

- Next, we create a `Connectivity` that stores connections between qubits on the target hardware. Two-qubit gates will only be allowed between connected qubits. For this example, we use a `"circular"` connectivity on `num_qubits=4` qubits. Other predefined connectivities can be queried via `htlg.available_connectivities()`. Moreover, a custom connectivity can be created from an adjacency matrix. 

- Finally, we initialize a `Circuit` with the number of logical qubits and add a Hadamard gate on the first qubit (note that we count **qubits starting at 0**). In the following we will tailor a circuit that implements the action of this circuit on the logical level of the stabilizer code.

Note that we created a `Connectivity` for $n=4$ qubits and a logical `Circuit` for $k=2$ qubits since we are using the $⟦n=4,k=2,2⟧$-code.

Now we can pass these objects to the function `tailor_logical_gate`:

In [3]:
circ, status = htlg.tailor_logical_gate(stab_code, connectivity, logical_gate, num_cz_layers=2)

The parameter `num_cz_layers` determines the number of CZ gate layers in the ansatz circuit. Generally speaking, more CZ layers make the ansatz more expressive and can lead to circuits with less two-qubit gates in total, while increasing runtime. If you can not find a specific gate, try to increase the number of CZ gate layers.

The return value `status` indicates the state of the optimization:

- `"Optimal"`: The returned circuit is optimal in terms of two-qubit gates.

- `"Bound {n}"`: The returned circuit is not optimal in terms of two-qubit games but there is no circuit with less than $n$ two-qubit gates.

- `"Infeasible"`: There is no physical circuit for the given stabilizer code, connectivity, logical circuit, and number of CZ gate layer.

- `"Time out"`: A physical circuit was not found in the given time limit.

If the status message is `"Optimal"` or `"Bound {n}"`, then `circ` contains the physical circuit implementation. Otherwise, it is `None`.

In [4]:
print(f'Status: "{status}"\n')
print(f'Circuit:\n{circ}')

Status: "Optimal"

Circuit:
Z 1
C_XYZ 3
CZ 1 2
CZ 2 3
C_XYZ 2
CZ 1 2
CZ 2 3
S 2
C_ZYX 3



## More ways to create a `StabilizerCode`

There exists a number of stabilizer codes that are already included in the package. A list of these can be accessed as follows:

In [5]:
htlg.available_stabilizercodes()

{'8_3_2': '[[8,3,2]] hypercube and color code.',
 '4_2_2': '[[4,2,2]] iceberg and color code.',
 '12_2_3': '[[12,2,3]] twisted toric-24 code.',
 'trivial': 'The trivial [[n,n,1]] code.'}

The keys of this dictionary, and sometimes an integer `n`, can be passed to the `StabilizerCode` constructor.

To create our own stabilizer codes, we can can create a list of logical Pauli-X, Pauli-Z, and stabilizer operators. Hereby, Pauli operators are represented as strings and the correctness of the stabilizer code is verified when creating the stabilizer code object.

In [6]:
x_logicals = ["X0 X1 X2 X3 X4"]
z_logicals = ["Z0 Z1 Z2 Z3 Z4"]
stabilizers = ["X0 Z1 Z2 X3",
               "X1 Z2 Z3 X4",
               "X0 X2 Z3 Z4",
               "Z0 X1 X3 Z4"]

stab_code = htlg.StabilizerCode(x_logicals, z_logicals, stabilizers)

## More ways to create a `Connectivity`

Some basic connectivities are included in the package. They can be queried as follows:

In [7]:
htlg.available_connectivities()

{'8_3_2_cube': 'Cube connectivity for the [[8,3,2]]-color code.',
 '12_2_3_checker': 'Checkerboard connectivity for the [[12,2,3]] twisted toric-24 code.',
 'linear': 'Connections exist between qubits $i$ and $i+1$.',
 'circular': 'Connections exist between qubits $i$ and $(i+1)%n$.',
 'all': 'Connections exist between all qubits.'}

The keys of this dictionary, and sometimes an integer `n`, can be passed to the `Connectivity` constructor.

Alternatively, we can create a `Connectivity` by passing an $n\times n$ adjacency matrix to the constructor.

In [10]:
import numpy as np

adjacency_matrix = np.array([
    [0, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0]
])
connectivity = htlg.Connectivity(adjacency_matrix)

## More ways to create a `Circuit`

The interface of the circuit class is inspired by other popular packages such as [qiskit](https://github.com/Qiskit/qiskit) or [stim](https://github.com/quantumlib/stim). A `Circuit` object can be created from a string or be build step-by-step. To create a circuit from a string, the string can be passed to the constructor (note that this string is not case-sensitive):

In [11]:
string = """
h 0
cz 0 1
h 1
c_xyz 0
"""
circ = htlg.Circuit(string)

The same circuit can also be build step-by-step:

In [12]:
circ = htlg.Circuit(2)
circ.h(0)
circ.cz(0, 1)
circ.h(1)
circ.c_xyz(0)

## Available gates

In the following, we provide a list of all gates and their transformations of stabilizer generators.

- `'id'` gate: `X → X, Z → Z`

- `'x'` gate: `X → X, Z → −Z`

- `'y'` gate: `X → −X, Z → −Z`

- `'z'` gate: `X → −X, Z → Z`

- `'h'` gate: `X → Z, Z → X`

- `'s'` gate: `X → Y, Z → Z`

- `'sdg'` gate: `X → −Y, Z → Z`

- `'sxdg'` gate: `X → X, Z → Y`

- `'c_xyz'` gate: `X → Y, Z → X`

- `'c_zyx'` gate: `X → Z, Z → Y`

- `'swap'` gate: `XI → IX, ZI → IZ, IX → XI, IZ → ZI`

- `'cx'` gate: `XI → XZ, ZI → ZI, IX → IX, IZ → XZ`

- `'cz'` gate: `XI → XZ, ZI → ZI, IX → ZX, IZ → IZ`